In [ ]:
import pandas as pd

# Assumes the patient data has columns 'DATE', 'PName', and 'Department'

def adjust_inventory(patient_data, medicine_data, stock_threshold=10):
    recommendations = []

    # Preprocessing patient data
    patient_data['DATE'] = pd.to_datetime(patient_data['DATE'])  # Ensuring DATE is datetime type
    patient_data['Month'] = patient_data['DATE'].dt.month_name()  # Extract month for analysis

    # Calculate monthly patient counts per department
    monthly_visits = patient_data.groupby(['Department', 'Month']).size().reset_index(name='Patient_Count')

    # Loop through each department in patient data
    for department in monthly_visits['Department'].unique():
        # Get monthly patient counts for the current department
        dept_patient_counts = monthly_visits[monthly_visits['Department'] == department]
        # Filter medicines used by the department (based on Category Name)
        dept_medicines = medicine_data[medicine_data['Category Name'] == department]

        # Calculate average monthly patient count for demand estimation
        avg_patient_count = dept_patient_counts['Patient_Count'].mean()

        # Loop through medicines and make stock adjustment recommendations
        for _, row in dept_medicines.iterrows():
            active_stock = row['Active Stock']
            medicine_name = row['Medicine Name']
            category = row['Category Name']

            # Adjust stock based on threshold or predicted demand
            if active_stock < stock_threshold:
                recommendation = (f"Recommendation: Increase stock of '{medicine_name}' in '{category}'. "
                                  f"Current active stock is {active_stock}, below the threshold of {stock_threshold}.")
                recommendations.append(recommendation)
            elif avg_patient_count > active_stock:
                recommendation = (f"Recommendation: Consider increasing '{medicine_name}' in '{category}' to meet demand. "
                                  f"Avg monthly patients: {avg_patient_count}, but active stock is {active_stock}.")
                recommendations.append(recommendation)
            else:
                recommendation = (f"Stock for '{medicine_name}' in '{category}' is adequate.")
                recommendations.append(recommendation)

    return recommendations

# Assuming `patient_data.csv` is the input file with columns ['DATE', 'PName', 'Department'] and `medicine_data` as previously defined
def main():
    # Load patient data
    patient_data = pd.read_csv('/content/patient_consolidated.csv')  # Replace 'patient_data.csv' with the actual file path

    # Sample medicine data as provided
    medicine_data = pd.DataFrame({
        'Medicine Name': ['TYNOR THUMB SPICA', '2 WAY FOLEYS BALLON CATHETER FG-16', 'A TO Z NS NEW STRIP OF 15 TABLETS',
                          'A TO Z NS STRAWBERRY FLAVOUR BOTTLE OF 15ML DROPS', 'AAPSON C FIT HINGED KNEE SUPPORT M'],
        'Category Name': ['GENERAL SURGERY', 'GENERAL MEDICINE', 'GENERAL MEDICINE', 'PED', 'ORTHO'],
        'Item Type': ['SPINLT', 'PACKET', 'PACKET', 'PACKET', 'BOX'],
        'Opening Stock': [0, 4, 1, 80, 0],
        'Purchase Qty': [4, 5, 0, 905, 23],
        'Closing Quantity': [2, 4, 0, 70, 5],
        'Active Stock': [2, 5, 1, 915, 18]
    })

    # Generate inventory adjustment recommendations
    recommendations = adjust_inventory(patient_data, medicine_data)

    # Print recommendations
    for rec in recommendations:
        print(rec)


main()